In [1]:
import os

In [2]:
%pwd

'/home/uday/Practise/github_personal/pothole_detection/pothole_detection/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/uday/Practise/github_personal/pothole_detection/pothole_detection'

In [5]:
from pathlib import Path
from dataclasses import dataclass

In [6]:
@dataclass(frozen=True)
class ValidationConfig:
    params_task: str
    params_mode: str
    params_model: Path
    params_imgsz: int
    # params_data: Path
    params_name: str
    params_project: Path

In [7]:
from potholeDetector.constants import *
from potholeDetector.utils.common import *

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
    
    def get_validation_config(self) -> ValidationConfig:
        # config_prepare_base_model = self.config.prepare_base_model
        config_validation = self.config.validation
        params = self.params.evaluation
        create_directories([
            self.config.artifacts_root, 
            config_validation.root_dir, 
            config_validation.validation_results_dir,
            config_validation.weights_dir
            ])
        validation_config = ValidationConfig(
            params_task=params.task,
            params_mode=params.mode,
            params_model=params.model,
            params_imgsz=params.imgsz,
            # params_data=Path(params.data),
            params_name=params.name, 
            params_project=params.project, 
        )

        return validation_config

In [9]:
from ultralytics import YOLO
import shutil
import os

In [18]:
class Validation:
    def __init__(self, config: ValidationConfig):
        self.config = config
    
    def val(self):
        # Load the YOLOv8 model.
        model = YOLO(self.config.params_model)


        # Evaluate the model.
        # yolo task=detect mode=val model=runs/detect/yolov8m_v8_50e/weights/best.pt name=yolov8m_eval data=pothole_v8.yaml imgsz=1280
        results = model.val(
            task=self.config.params_task,
            mode=self.config.params_mode,
            # model=self.config.params_model,
            imgsz=self.config.params_imgsz,
            # data=self.config.params_data,
            name=self.config.params_name,
            project=self.config.params_project,
        )
        
    def copy_final_model_weight(self, proj_config):
        """
        Copy the final model weight file to the specified destination directory.

        This function takes the source path of the final model weight file and the
        destination directory path from the project configuration.

        Args:
            proj_config (ProjectConfig): An instance of the project configuration class.

        Returns:
            None
        """
        # Get the source path of the final model weight file from params
        source_path = self.config.params_model
        
        # Get the destination directory from the project configuration
        destination_directory = proj_config.config.validation.weights_dir
        
        # Check if the source file and destination directory exist
        if os.path.isfile(source_path) and os.path.isdir(destination_directory):
            # Copy the source file to the destination directory with a specific name
            destination_path = os.path.join(destination_directory, 'final_weight.pt')
            shutil.copy(source_path, destination_path)
            print(f"Final model weight file copied to {destination_path}")
        else:
            print("Error: Either the source file or the destination directory does not exist.")




In [19]:
try:
    config = ConfigurationManager()
    validation_config = config.get_validation_config()
    print(validation_config)
    validation = Validation(config=validation_config)
    result = validation.val()
    print(result)
    validation.copy_final_model_weight(config)
except Exception as e:
    raise e

[2023-08-20 11:46:23,861: INFO:common:YAML file: config/config.yaml loaded successfully]
[2023-08-20 11:46:23,865: INFO:common:YAML file: params.yaml loaded successfully]
[2023-08-20 11:46:23,867: INFO:common:Created directory at: artifacts]
[2023-08-20 11:46:23,867: INFO:common:Created directory at: artifacts/validation]
[2023-08-20 11:46:23,868: INFO:common:Created directory at: artifacts/validation/runs]
[2023-08-20 11:46:23,868: INFO:common:Created directory at: artifacts/validation/weights]
ValidationConfig(params_task='detect', params_mode='val', params_model='artifacts/training/runs/yolov8n_v8_50e/weights/best.pt', params_imgsz=1280, params_name='yolov8n_eval', params_project='/home/uday/Practise/github_personal/pothole_detection/pothole_detection/artifacts/validation/runs')


Ultralytics YOLOv8.0.135 🚀 Python-3.10.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients
val: Scanning /home/uday/Practise/github_personal/pothole_detection/pothole_detection/artifacts/data_ingestion/dataset/valid/labels.cache... 271 images, 0 backgrounds, 0 corrupt: 100%|██████████| 271/271 [00:00<?, ?it/s]
val: WARNING ⚠️ /home/uday/Practise/github_personal/pothole_detection/pothole_detection/artifacts/data_ingestion/dataset/valid/images/G0011603.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /home/uday/Practise/github_personal/pothole_detection/pothole_detection/artifacts/data_ingestion/dataset/valid/images/G0011614.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /home/uday/Practise/github_personal/pothole_detection/pothole_detection/artifacts/data_ingestion/dataset/valid/images/G0011616.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /home/uday/Practise/github_personal/pothole_detection/potho

None
Final model weight file copied to artifacts/validation/weights/final_weight.pt
